In [1]:
from scipy.io import netcdf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import netCDF4
import os
import tensorflow as tf
import glob
import pandas as pd

2022-04-20 18:48:34.539619: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-20 18:48:34.539666: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
def cross_corr(ts0, ts1):
    return np.sum(ts0 * ts1) / (np.linalg.norm(ts0) * np.linalg.norm(ts1))

In [2]:
path_data_train = '/home/user/Downloads/arctic_data/2012'
path_data_test = '/home/user/Downloads/arctic_data/2013'

In [15]:
a = setting_nc.variables['icethic_cea'][:, predict_point[0]-1:predict_point[0]+2, predict_point[1]-1:predict_point[1]+2].data
a.shape

(24, 3, 3)

In [3]:
setting_nc = netCDF4.Dataset(os.path.join(path_data_train, 'ARCTIC_1h_ice_grid_TUV_20120101-20120101.nc'))
setting_nc_shp = np.array(setting_nc.variables['nav_lon']).shape
predict_point = (setting_nc_shp[0] // 2, setting_nc_shp[1] // 2)
predict_point

(226, 203)

In [14]:
ts_train = []
for filename in glob.glob(os.path.join(path_data_test, '*.nc')):
    nc_file = netCDF4.Dataset(str(filename))
    target_feature = nc_file.variables['icethic_cea'][:, predict_point[0]-1:predict_point[0]+2, predict_point[1]-1:predict_point[1]+2].data
    try:
        ts_train = np.vstack((ts_train, target_feature))
    except:
        ts_train = target_feature.copy()

In [15]:
ts_train.shape

(8760, 3, 3)

In [8]:
corr_result = np.zeros((ts_train.shape[1], ts_train.shape[2]))
for i in range(ts_train.shape[1]):
    for j in range(ts_train.shape[2]):
        corr_result[i][j] = cross_corr(ts_train[:, 1, 1], ts_train[:, i, j])

In [11]:
corr_result

array([[0.99985623, 0.99983948, 0.99973708],
       [0.99991161, 1.        , 0.99983829],
       [0.99976689, 0.99974585, 0.99984926]])

In [13]:
np.savetxt("/home/user/Downloads/arctic_results/ts_test.csv", ts_train.reshape((9, ts_train.shape[0])), delimiter=",")